<a href="https://colab.research.google.com/github/Busraayuksel/ToDoList/blob/master/multi_class_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install simpletransformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import string
import warnings
# from simpletransformers.classification import MultiLabelClassificationModel
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data = pd.read_excel(r"/content/drive/MyDrive/yeni_birlesmis_veriler2.xlsx")
df = pd.DataFrame(data)
pd.set_option("max_colwidth",300)
df.head(2)

,Author Full Names,Article Title,Source Title,Document Type,Author Keywords,Keywords Plus,Abstract,Affiliations,Cited References,Cited Reference Count,"Times Cited, WoS Core","Times Cited, All Databases",ISSN,eISSN,WoS Categories,Research Areas
0,"Jiang, Weijiao; Sun, Chuandong; Zhang, Yue; Xie, Zhihui; Zhou, Jin; Kang, Jian; Cao, Ya; Xiang, Ming",Preparation of well-dispersed graphene oxide-silica nanohybrids/poly (lactic acid) composites by melt mixing,POLYMER TESTING,Article,Graphene oxide-silica nanohybrids; poly(lactic acid); Melt blending; Properties,POLY(LACTIC ACID); CRYSTALLIZATION; BEHAVIORS,"Graphene oxide has a strong tendency to agglomerate in polymers that are only hydrogen bonded acceptors due to the strong hydrogen bonding and van der Waals forces between its lamellae, which greatly limits its appli-cation. In this work, Silica (SiO2) was introduced on the surface of graphene o...",Sichuan University; Sichuan University,"Balandin AA, 2008, NANO LETT, V8, P902, DOI 10.1021/nl0731872; Cai DY, 2010, J MATER CHEM, V20, P7906, DOI 10.1039/c0jm00530d; Cheng ZL, 2017, J ALLOY COMPD, V722, P778, DOI 10.1016/j.jallcom.2017.06.161; de Melo-Diogo D, 2019, BIOMATER SCI-UK, V7, P3534, DOI 10.1039/c9bm00577c; Dreyer DR, 2010,...",35,2,2,0142-9418,1873-2348,"Materials Science, Characterization & Testing; Polymer Science",Materials Science; Polymer Science
1,"Jiao, Xinyang; Zhang, Yu; Wu, Qingli; Shang, Zhichao; Yu, Yang; Zhang, Xiaoxiao; Feng, Peizhong",High-temperature oxidation behavior and pore structure of porous TiAl3 intermetallics at 650?C to 900?C,JOURNAL OF MATERIALS RESEARCH AND TECHNOLOGY-JMR&T,Article,Porous materials; Thermal explosion; Sintering; Microstructure; Oxidation resistance,THERMAL-EXPLOSION; REACTIVE SYNTHESIS; MICROSTRUCTURE; FABRICATION; RESISTANCE; DIFFUSION; POROSITY; ALLOYS,"Porous TiAl3 materials were prepared by a time-and energy-saving thermal explosion (TE) method with Ti-75 at% Al mixture powders. Cyclic oxidation tests were performed in air at 650 degrees C, 800 degrees C, and 900 degrees C. Only a weak peak of the oxide phase was detected by X-ray diffraction...",Suqian University; Suqian University; China University of Mining & Technology; University of New South Wales Sydney,"Alimadadi H, 2017, MATER CHARACT, V130, P105, DOI 10.1016/j.matchar.2017.05.039; BOLT PH, 1995, SURF SCI, V329, P227, DOI 10.1016/0039-6028(95)00063-1; Cai XP, 2021, CORROS SCI, V191, DOI 10.1016/j.corsci.2021.109731; Cai XP, 2019, J MATER RES TECHNOL, V8, P3188, DOI 10.1016/j.jmrt.2019.05.007; ...",31,0,0,2238-7854,2214-0697,"Materials Science, Multidisciplinary; Metallurgy & Metallurgical Engineering",Materials Science; Metallurgy & Metallurgical Engineering


In [ ]:
df.shape

(100998, 16)

In [ ]:
df.dtypes

Author Full Names             object
Article Title                 object
Source Title                  object
Document Type                 object
Author Keywords               object
Keywords Plus                 object
Abstract                      object
Affiliations                  object
Cited References              object
Cited Reference Count          int64
Times Cited, WoS Core          int64
Times Cited, All Databases     int64
ISSN                          object
eISSN                         object
WoS Categories                object
Research Areas                object
dtype: object

In [ ]:
# Araştırma alanları değişkenindeki değerler
pd.DataFrame(df["Research Areas"].unique())

,0
0,Materials Science; Polymer Science
1,Materials Science; Metallurgy & Metallurgical Engineering
2,Construction & Building Technology; Engineering; Materials Science
3,Oncology
4,Engineering; Materials Science
...,...
332,Biochemistry & Molecular Biology; Mathematical & Computational Biology; Mathematics
333,Thermodynamics; Energy & Fuels; Engineering; Materials Science
334,Construction & Building Technology; Science & Technology - Other Topics; Materials Science
335,Chemistry; Food Science & Technology; Immunology; Toxicology


In [ ]:
#337 farklı kategori var
df["Research Areas"].nunique()

337

#DATA CLEANING

In [ ]:
# veriyi karıştırma
df = df.sample(frac=1)
df.head(2)

,Author Full Names,Article Title,Source Title,Document Type,Author Keywords,Keywords Plus,Abstract,Affiliations,Cited References,Cited Reference Count,"Times Cited, WoS Core","Times Cited, All Databases",ISSN,eISSN,WoS Categories,Research Areas
69142,"Ji, Yanhong; Miao, Peng; Zhang, Ruonan; Li, Quan; Zha, Yifan; Dai, Fengying; Du, Xi; He, Benqiao; Fu, Shaoyun",Novel epoxy adhesives based on conductive micron-sized silver flowers,POLYMER COMPOSITES,Article,electrical resistance; electrically conductive adhesive; lap shear strength; micron flower-like silver particles,PERCOLATION-THRESHOLD; COMPOSITES; COPPER; FLAKES; BEHAVIOR,"Low silver contents are desirable for electrically conductive adhesives (ECAs) due to the high cost of silver. In the present work, micro-sized flower-like silver particles (MFSPs) with long and thin petals are prepared by a novel, green, and facile chemical reduction method in an aqueous soluti...",Tiangong University; Tiangong University; Chongqing University,"Anuar SK, 2011, J MATER SCI-MATER EL, V22, P757, DOI 10.1007/s10854-010-0207-7; Chee SS, 2012, ELECTRON MATER LETT, V8, P315, DOI 10.1007/s13391-012-1105-3; Chen XG, 2021, POLYM COMPOSITE, V42, P5952, DOI 10.1002/pc.26276; Cui X., 2013, 14 INT C EL PACK TEC; Du CJ, 2019, J MATER SCI-MATER EL, V3...",28,4,4,0272-8397,1548-0569,"Materials Science, Composites; Polymer Science",Materials Science; Polymer Science
86713,"Huang, Boxin; He, Yong; Rofaani, Elrade; Liang, Feng; Huang, Xiaochen; Shi, Jian; Wang, Li; Yamada, Ayako; Peng, Juan; Chen, Yong",Automatic differentiation of human induced pluripotent stem cells toward synchronous neural networks on an arrayed monolayer of nanofiber membrane,ACTA BIOMATERIALIA,Article,hiPSCs; Cortical neural networks; Self -organization; Automatic differentiation,HUMAN BRAIN-DEVELOPMENT; ELECTROSPUN NANOFIBERS; FUNCTIONAL NETWORKS; NEURONAL NETWORKS; ORGANOIDS; DYNAMICS; MODEL; FABRICATION; PHYSIOLOGY; INDUCTION,"Automatic differentiation of human-induced pluripotent stem cells (hiPSCs) facilitates the generation of cortical neural networks and studies of brain functions. Here, we present a method of directed differ-entiation of hiPSCs with a substrate made of a honeycomb microframe and a monolayer of cr...",Universite PSL; Ecole Normale Superieure (ENS); Centre National de la Recherche Scientifique (CNRS); Sorbonne Universite,"Adil MM, 2018, ADV FUNCT MATER, V28, DOI 10.1002/adfm.201804144; de Sousa MCA, 2020, FRONT BIOENG BIOTECH, V8, DOI 10.3389/fbioe.2020.580135; Ankam S, 2013, ACTA BIOMATER, V9, P4535, DOI 10.1016/j.actbio.2012.08.018; Aregueta-Robles UA, 2019, ACTA BIOMATER, V95, P269, DOI 10.1016/j.actbio.2018.1...",81,3,3,1742-7061,1878-7568,"Engineering, Biomedical; Materials Science, Biomaterials",Engineering; Materials Science


## 1.Duplicates Values

In [ ]:
# Makale başlığı ve yazar adına göre aynı olan yani tekrar edenleri bulma.
df[df.duplicated(subset=['Article Title', 'Author Full Names'])][['Article Title', 'Author Full Names']].nunique()

Article Title        26
Author Full Names    26
dtype: int64

In [ ]:
# Tekrar edenlerden bir tanesi hariç diğerleri silinir.
df.drop_duplicates(subset = ['Article Title', 'Author Full Names'],keep = "first", inplace =True)

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.shape

(100972, 16)

## 2.Columns Remove

In [ ]:
# bu proje için makale başlığı, Araştırma Alanları ve özet alınmalı
# df["Abstract"] = df["Article Title"] + '. ' + df["Abstract"]
columns_to_keep = ["Article Title","Abstract", "Author Keywords",	"Keywords Plus", "Research Areas"]
df = df[columns_to_keep]
df.head(2)

,Article Title,Abstract,Author Keywords,Keywords Plus,Research Areas
69142,Novel epoxy adhesives based on conductive micron-sized silver flowers,"Low silver contents are desirable for electrically conductive adhesives (ECAs) due to the high cost of silver. In the present work, micro-sized flower-like silver particles (MFSPs) with long and thin petals are prepared by a novel, green, and facile chemical reduction method in an aqueous soluti...",electrical resistance; electrically conductive adhesive; lap shear strength; micron flower-like silver particles,PERCOLATION-THRESHOLD; COMPOSITES; COPPER; FLAKES; BEHAVIOR,Materials Science; Polymer Science
86713,Automatic differentiation of human induced pluripotent stem cells toward synchronous neural networks on an arrayed monolayer of nanofiber membrane,"Automatic differentiation of human-induced pluripotent stem cells (hiPSCs) facilitates the generation of cortical neural networks and studies of brain functions. Here, we present a method of directed differ-entiation of hiPSCs with a substrate made of a honeycomb microframe and a monolayer of cr...",hiPSCs; Cortical neural networks; Self -organization; Automatic differentiation,HUMAN BRAIN-DEVELOPMENT; ELECTROSPUN NANOFIBERS; FUNCTIONAL NETWORKS; NEURONAL NETWORKS; ORGANOIDS; DYNAMICS; MODEL; FABRICATION; PHYSIOLOGY; INDUCTION,Engineering; Materials Science


## 3.NULL Values

In [ ]:
df.isnull().sum()

Article Title          0
Abstract             366
Author Keywords    20111
Keywords Plus       8841
Research Areas         0
dtype: int64

In [ ]:
# özette boş olan değerleri sil
df.dropna(subset = ["Abstract"], inplace = True)

In [ ]:
df.isnull().sum()

Article Title          0
Abstract               0
Author Keywords    19871
Keywords Plus       8692
Research Areas         0
dtype: int64

In [ ]:
# df = df[~((df["Author Keywords"].isnull()) & (df["Keywords Plus"].isnull()))]

In [ ]:
df = df.fillna(value = {"Author Keywords": " ", "Keywords Plus" : " "})
df.isnull().sum()

Article Title      0
Abstract           0
Author Keywords    0
Keywords Plus      0
Research Areas     0
dtype: int64

In [ ]:
df["Article Title"] = df["Article Title"].astype(str)
df["Abstract"] = df["Abstract"].astype(str)
df["Abstract"] = df["Article Title"] + '. ' + df["Abstract"]

# Data Preprocessing For Categori

## 1. Lower case



In [ ]:
df["Abstract"] = df["Abstract"].apply(lambda x: x.lower())

## 2.Tokenization

In [ ]:
# her bir alan ; ve , ile ayrılmış. virgülleri noktalı virgüle çevirip noktalı virgüle göre alanları al
df["Research Areas"] = df["Research Areas"].replace(",",";")
alanlar= df["Research Areas"].apply(lambda x: x.split(';'))
alanlar.head(2)

69142    [Materials Science,  Polymer Science]
86713        [Engineering,  Materials Science]
Name: Research Areas, dtype: object

In [ ]:
# farklı alanları listeye al
unique_alanlar = []

for alan_listesi in alanlar:
    for alan in alan_listesi:
        alan = alan.strip()
        if alan not in unique_alanlar:
            unique_alanlar.append(alan)
unique_alanlar

['Materials Science',
 'Polymer Science',
 'Engineering',
 'Chemistry',
 'Metallurgy & Metallurgical Engineering',
 'Physics',
 'Agriculture',
 'Food Science & Technology',
 'Mathematics',
 'Science & Technology - Other Topics',
 'Veterinary Sciences',
 'Zoology',
 'Oncology',
 'Biochemistry & Molecular Biology',
 'Instruments & Instrumentation',
 'Crystallography',
 'Construction & Building Technology',
 'Cell Biology',
 'Acoustics',
 'Immunology',
 'Optics',
 'Radiology, Nuclear Medicine & Medical Imaging',
 'Neurosciences & Neurology',
 'Nursing',
 'Nutrition & Dietetics',
 'Biomedical Social Sciences',
 'Hematology',
 'Nuclear Science & Technology',
 'Environmental Sciences & Ecology',
 'Mathematical & Computational Biology',
 'Psychology',
 'Behavioral Sciences',
 'Archaeology',
 'Art',
 'Geology',
 'Spectroscopy',
 'Computer Science',
 'Emergency Medicine',
 'Respiratory System',
 'Research & Experimental Medicine',
 'Pharmacology & Pharmacy',
 'Energy & Fuels',
 'Operations Rese

In [ ]:
# kaç farklı alan var
len(unique_alanlar)

110

In [ ]:
# her bir kategoride kaç veri var
count_series = df['Research Areas'].str.split('; ').explode().value_counts()
count_series

Chemistry              47311
Materials Science      32341
Physics                18720
Mathematics            17245
Engineering            13818
                       ...  
Infectious Diseases        2
Paleontology               2
Substance Abuse            1
Government & Law           1
Sociology                  1
Name: Research Areas, Length: 110, dtype: int64

In [ ]:
# bu alanlardan sadece Mathematics, Chemistry, Zoology, Nursing, Metallurgy & Metallurgical Engineering ve Oncology alınacak
count_series = count_series[count_series.index.isin(["Mathematics", "Chemistry", "Zoology", "Nursing", "Metallurgy & Metallurgical Engineering", "Oncology"])]
count_series

Chemistry                                 47311
Mathematics                               17245
Oncology                                  13616
Metallurgy & Metallurgical Engineering     5392
Zoology                                    3129
Nursing                                    1946
Name: Research Areas, dtype: int64

In [ ]:
# count_series.describe()

In [ ]:
# # verilerin azlığından kaynaklı %50 değerinden az verinin olduğu alanı sil
# count_series = count_series[count_series >= np.round(count_series.describe()["50%"])]
# count_series

In [ ]:
# # kaç alan kaldı
# len(count_series)

In [ ]:
df['Research Areas'] = df['Research Areas'].apply(lambda x: x.split("; "))
df['Research Areas'] = df['Research Areas'].apply(lambda x: [item for item in x if item in count_series.index])

df['Research Areas'].head()

69142                                                     []
86713                                                     []
15699    [Chemistry, Metallurgy & Metallurgical Engineering]
93269                                            [Chemistry]
57674                                          [Mathematics]
Name: Research Areas, dtype: object

In [ ]:
# seçilen alanlar dışında olan verileri silme
def areas(area):
  if (len(area) == 1) and any(count_series.index.isin(area) == True):
    return area
  else:
    return None

df['Research Areas'] = df['Research Areas'].apply(areas)
df.dropna(inplace = True)
df['Research Areas']

93269      [Chemistry]
57674    [Mathematics]
87470      [Chemistry]
26059    [Mathematics]
19163      [Chemistry]
             ...      
63543      [Chemistry]
61432      [Chemistry]
12389      [Chemistry]
10816        [Zoology]
53896       [Oncology]
Name: Research Areas, Length: 81527, dtype: object

## 3.Category Labelling

In [ ]:
# her bir alana unique ıd ver.
ids = [i for i in range(1,len(count_series.index)+1)]
id_alan = dict(zip(ids, count_series.index))
id_alan

{1: 'Chemistry',
 2: 'Mathematics',
 3: 'Oncology',
 4: 'Metallurgy & Metallurgical Engineering',
 5: 'Zoology',
 6: 'Nursing'}

In [ ]:
# araştırma alanları değerini bu ıd lerle değiştir ve yeni bir kolona ata
def alanlari_degistir(alanlar):
    alan_idleri = []
    for alan in alanlar:
        for k,v in id_alan.items():
            if alan == v:
                alan_idleri.append(k)
    return alan_idleri

df['label Research Areas'] = df['Research Areas'].apply(alanlari_degistir)
df

,Article Title,Abstract,Author Keywords,Keywords Plus,Research Areas,label Research Areas
93269,Atypical ageing defect in Pinot Blanc wines: a comparison between organic and conventional production management systems,"atypical ageing defect in pinot blanc wines: a comparison between organic and conventional production management systems. background atypical ageing (ata) is an aroma defect that occurs in white wines and entails a loss of varietal aromas as well as scents of wet mop, shoe polish and dish rag. 2...",atypical ageing; Pinot Blanc; UHPLC-HRMS; aroma fault; 2-aminoacetophenone; organic wine,BIOGENIC-AMINES; OFF-FLAVOR; GRAPE; GLUTATHIONE; TRYPTOPHAN; 2-AMINOACETOPHENONE; PROLINE; STABILITY; SOLUTES; PLANTS,[Chemistry],[1]
57674,Investigations of isotropy and homogeneity of spacetime in first-order logic,"investigations of isotropy and homogeneity of spacetime in first-order logic. we investigate the logical connection between (spatial) isotropy, homogeneity of space, and homogeneity of time within a general axiomatic framework. we show that isotropy not only entails homogeneity of space, but als...",First-order logic; Relativity theory; Classical spacetime; Homogeneity; Isotropy; Axiomatisation,RELATIVITY,[Mathematics],[2]
87470,Influence of Electrochemical Anticorrosion Technology on Concrete Structure and Performance,"influence of electrochemical anticorrosion technology on concrete structure and performance. in order to solve the problem of steel bar desensitization in island concrete, which leads to steel bar corrosion and greatly reduces the service life of structures, this work studies the in+uence of ele...",,DURABILITY; CORROSION; DESIGN; BARS,[Chemistry],[1]
26059,Invariance of Plurigenera Fails in Positive and Mixed Characteristic,"invariance of plurigenera fails in positive and mixed characteristic. we construct families of elliptic surface pairs with terminal singularities over a dvr of positive or mixed characteristic (x, b) -> specr, such that p-m(x-k, b-k) > p-m(x-k, b-k) for all sufficiently divisible m > 0. in parti...",,SURFACES; VARIETIES,[Mathematics],[2]
19163,"In Situ Generated DBU•HF Acts as a Fluorinating Agent in a Hexafluoroisobutylation Tandem Reaction: An Effective Route to 5,5,5,5′,5′,5′-Hexafluoroleucine","in situ generated dbu•hf acts as a fluorinating agent in a hexafluoroisobutylation tandem reaction: an effective route to 5,5,5,5′,5′,5′-hexafluoroleucine. we report the direct incorporation of the hexafluoroisobutyl group on a chiral glycine schiff base complex mediated by 1,8-diazabicyclo[5.4....",fluoroalkylation; fluorination; amino acids; stereoselective synthesis; synthetic methods,"ALPHA-AMINO-ACIDS; FUNCTIONALIZED 1,1-DIFLUORO-1-ALKENES; ORGANOLITHIUM REAGENTS; PROTEOLYTIC STABILITY; ASYMMETRIC-SYNTHESIS; GEM-DIFLUOROALKENES; BIOLOGICAL-ACTIVITY; CONCISE SYNTHESIS; CHEMISTRY; HOMOLOGATION",[Chemistry],[1]
...,...,...,...,...,...,...
63543,Boosting Bismuth(III) Complexation for Targeted α-Therapy (TAT) Applications with the Mesocyclic Chelating Agent AAZTA,"boosting bismuth(iii) complexation for targeted α-therapy (tat) applications with the mesocyclic chelating agent aazta. targeted alpha therapy (tat) is a promising tool in the therapy of cancer. the radionuclide bi-213(iii) shows favourable physical properties for this application, but the fast ...",AAZTA; Bioconjugation; Bismuth; Octreotate; Targeted Alpha Therapy,NEUROENDOCRINE TUMORS; X-RAY; LIGANDS; PET; EQUILIBRIUM; STABILITY; ANALOGS; GA-68; SC-44; NMR,[Chemistry],[1]
61432,A Data-Driven OBE Magnetic Interference Compensation Method,"a data-driven obe magnetic interference compensation method. aeromagnetic compensation is a technology used to reduce aircraft magnetic interference, which plays an important role in aeromagnetic surveys. in addition to maneuvering interferences, the onboard electronic (obe) interference has bee...",aeromagnetic survey; OBE interference compensation; LSTM network,MODEL; NETWORKS,[Chemistry]

In [ ]:
df.drop(columns=["Article Title", "Research Areas"], inplace = True)

In [ ]:
# # veri sızmasını önlemek için veri setini bölme
# train, test = train_test_split(
#     df,
#     test_size=0.2,
#     random_state=42,
#     shuffle= True,
#     stratify= df["label Research Areas"]
# )

# Abstract, Author Keywords, Keywords Plus Preprocessing

## 1.Lower

In [ ]:
# df["Abstract"] = df["Abstract"].apply(lambda x: str(x).lower())
# df["Abstract"]
for column in ["Abstract", "Author Keywords", "Keywords Plus"]:
  df[column] = df[column].apply(lambda x: str(x).lower())

df

,Abstract,Author Keywords,Keywords Plus,label Research Areas
93269,"atypical ageing defect in pinot blanc wines: a comparison between organic and conventional production management systems. background atypical ageing (ata) is an aroma defect that occurs in white wines and entails a loss of varietal aromas as well as scents of wet mop, shoe polish and dish rag. 2...",atypical ageing; pinot blanc; uhplc-hrms; aroma fault; 2-aminoacetophenone; organic wine,biogenic-amines; off-flavor; grape; glutathione; tryptophan; 2-aminoacetophenone; proline; stability; solutes; plants,[1]
57674,"investigations of isotropy and homogeneity of spacetime in first-order logic. we investigate the logical connection between (spatial) isotropy, homogeneity of space, and homogeneity of time within a general axiomatic framework. we show that isotropy not only entails homogeneity of space, but als...",first-order logic; relativity theory; classical spacetime; homogeneity; isotropy; axiomatisation,relativity,[2]
87470,"influence of electrochemical anticorrosion technology on concrete structure and performance. in order to solve the problem of steel bar desensitization in island concrete, which leads to steel bar corrosion and greatly reduces the service life of structures, this work studies the in+uence of ele...",,durability; corrosion; design; bars,[1]
26059,"invariance of plurigenera fails in positive and mixed characteristic. we construct families of elliptic surface pairs with terminal singularities over a dvr of positive or mixed characteristic (x, b) -> specr, such that p-m(x-k, b-k) > p-m(x-k, b-k) for all sufficiently divisible m > 0. in parti...",,surfaces; varieties,[2]
19163,"in situ generated dbu•hf acts as a fluorinating agent in a hexafluoroisobutylation tandem reaction: an effective route to 5,5,5,5′,5′,5′-hexafluoroleucine. we report the direct incorporation of the hexafluoroisobutyl group on a chiral glycine schiff base complex mediated by 1,8-diazabicyclo[5.4....",fluoroalkylation; fluorination; amino acids; stereoselective synthesis; synthetic methods,"alpha-amino-acids; functionalized 1,1-difluoro-1-alkenes; organolithium reagents; proteolytic stability; asymmetric-synthesis; gem-difluoroalkenes; biological-activity; concise synthesis; chemistry; homologation",[1]
...,...,...,...,...
63543,"boosting bismuth(iii) complexation for targeted α-therapy (tat) applications with the mesocyclic chelating agent aazta. targeted alpha therapy (tat) is a promising tool in the therapy of cancer. the radionuclide bi-213(iii) shows favourable physical properties for this application, but the fast ...",aazta; bioconjugation; bismuth; octreotate; targeted alpha therapy,neuroendocrine tumors; x-ray; ligands; pet; equilibrium; stability; analogs; ga-68; sc-44; nmr,[1]
61432,"a data-driven obe magnetic interference compensation method. aeromagnetic compensation is a technology used to reduce aircraft magnetic interference, which plays an important role in aeromagnetic surveys. in addition to maneuvering interferences, the onboard electronic (obe) interference has bee...",aeromagnetic survey; obe interference compensation; lstm network,model; networks,[1]
12389,"allosteric activation of 15-lipoxygenase-1 by boswellic acid induces the lipid mediator class switch to promote resolution of inflammation. specialized pro-resolving mediators (spm), primarily produced in innate immune cells, exert crucial bioactions for resolving inflammation. among various lip...",boswellic acid; inflammation; lipid mediators; lipoxygenases,leukotriene biosynthesis; 5-lipoxygenase-activating protein; in-vitro; disease; calcium; inhibitors; biology; domain; vivo,[1]
10816,"microbiota community structure and interaction networks within dermacentor silvarum, ixodes persulcatus, and haemaphysalis concinna. simple summary ticks pose a threat to humans and animals. however, microbial interactions on ticks are underappreciated. complex microbial interactions shape micro...",tick; micro

## 2.Contractions

In [ ]:
# I'll gibi yerleri I will yapma işlemi.
import json
def replace_contractions(text):
  with open(r"/content/drive/MyDrive/contractions_dict.json", "r") as f:
    contractions_dict= json.load(f)
    for key, value in contractions_dict.items():
        text = text.replace(key, value)
    return text

for column in ["Abstract", "Author Keywords", "Keywords Plus"]:
  df[column] = df[column].apply(replace_contractions)

# 3. Removing the Punctuations, Digits, URL and Time-Date

In [ ]:
# urlleri kaldırıyoruz
df["Abstract"] = df["Abstract"].apply(lambda x: re.sub(r'http[s]?://', '', x))

In [ ]:
# () içindeki kısaltmaları ve parantezleri kaldırıyoruz. Örnek olarak epidermal inclusion cysts (eic) ancak () içindekiler cümle ise sadece parantezler silinecek
for column in ["Abstract", "Author Keywords", "Keywords Plus"]:
  df[column] = df[column].apply(lambda x: re.sub(r'\([^)]*\)', ' ', x))
df

,Abstract,Author Keywords,Keywords Plus,label Research Areas
93269,"atypical ageing defect in pinot blanc wines: a comparison between organic and conventional production management systems. background atypical ageing is an aroma defect that occurs in white wines and entails a loss of varietal aromas as well as scents of wet mop, shoe polish and dish rag. 2-ami...",atypical ageing; pinot blanc; uhplc-hrms; aroma fault; 2-aminoacetophenone; organic wine,biogenic-amines; off-flavor; grape; glutathione; tryptophan; 2-aminoacetophenone; proline; stability; solutes; plants,[1]
57674,"investigations of isotropy and homogeneity of spacetime in first-order logic. we investigate the logical connection between isotropy, homogeneity of space, and homogeneity of time within a general axiomatic framework. we show that isotropy not only entails homogeneity of space, but also, in ce...",first-order logic; relativity theory; classical spacetime; homogeneity; isotropy; axiomatisation,relativity,[2]
87470,"influence of electrochemical anticorrosion technology on concrete structure and performance. in order to solve the problem of steel bar desensitization in island concrete, which leads to steel bar corrosion and greatly reduces the service life of structures, this work studies the in+uence of ele...",,durability; corrosion; design; bars,[1]
26059,"invariance of plurigenera fails in positive and mixed characteristic. we construct families of elliptic surface pairs with terminal singularities over a dvr of positive or mixed characteristic -> specr, such that p-m > p-m for all sufficiently divisible m > 0. in particular, this shows that ...",,surfaces; varieties,[2]
19163,"in situ generated dbu•hf acts as a fluorinating agent in a hexafluoroisobutylation tandem reaction: an effective route to 5,5,5,5′,5′,5′-hexafluoroleucine. we report the direct incorporation of the hexafluoroisobutyl group on a chiral glycine schiff base complex mediated by 1,8-diazabicyclo[5.4....",fluoroalkylation; fluorination; amino acids; stereoselective synthesis; synthetic methods,"alpha-amino-acids; functionalized 1,1-difluoro-1-alkenes; organolithium reagents; proteolytic stability; asymmetric-synthesis; gem-difluoroalkenes; biological-activity; concise synthesis; chemistry; homologation",[1]
...,...,...,...,...
63543,"boosting bismuth complexation for targeted α-therapy applications with the mesocyclic chelating agent aazta. targeted alpha therapy is a promising tool in the therapy of cancer. the radionuclide bi-213 shows favourable physical properties for this application, but the fast and stable chela...",aazta; bioconjugation; bismuth; octreotate; targeted alpha therapy,neuroendocrine tumors; x-ray; ligands; pet; equilibrium; stability; analogs; ga-68; sc-44; nmr,[1]
61432,"a data-driven obe magnetic interference compensation method. aeromagnetic compensation is a technology used to reduce aircraft magnetic interference, which plays an important role in aeromagnetic surveys. in addition to maneuvering interferences, the onboard electronic interference has been pr...",aeromagnetic survey; obe interference compensation; lstm network,model; networks,[1]
12389,"allosteric activation of 15-lipoxygenase-1 by boswellic acid induces the lipid mediator class switch to promote resolution of inflammation. specialized pro-resolving mediators , primarily produced in innate immune cells, exert crucial bioactions for resolving inflammation. among various lipoxyg...",boswellic acid; inflammation; lipid mediators; lipoxygenases,leukotriene biosynthesis; 5-lipoxygenase-activating protein; in-vitro; disease; calcium; inhibitors; biology; domain; vivo,[1]
10816,"microbiota community structure and interaction networks within dermacentor silvarum, ixodes persulcatus, and haemaphysalis concinna. simple summary ticks pose a threat to humans and animals. however, microbial interactions on ticks are underappreciated. complex microbial interactions shape micro...",tick; microbiota; microbial 

In [ ]:
# tarih ve saatleri kaldırma
def remove_time_and_dates(text):
    for match in re.finditer(r"\d{1,2}:\d{2}(:\d{2})? [AP]M|[0-9]{4}-[0-9]{2}-[0-9]{2}", text):
        start = match.start()
        end = match.end()
        text = text[:start] + text[end:]
    return text

df["Abstract"] = df["Abstract"].apply(remove_time_and_dates)
df

,Abstract,Author Keywords,Keywords Plus,label Research Areas
93269,"atypical ageing defect in pinot blanc wines: a comparison between organic and conventional production management systems. background atypical ageing is an aroma defect that occurs in white wines and entails a loss of varietal aromas as well as scents of wet mop, shoe polish and dish rag. 2-ami...",atypical ageing; pinot blanc; uhplc-hrms; aroma fault; 2-aminoacetophenone; organic wine,biogenic-amines; off-flavor; grape; glutathione; tryptophan; 2-aminoacetophenone; proline; stability; solutes; plants,[1]
57674,"investigations of isotropy and homogeneity of spacetime in first-order logic. we investigate the logical connection between isotropy, homogeneity of space, and homogeneity of time within a general axiomatic framework. we show that isotropy not only entails homogeneity of space, but also, in ce...",first-order logic; relativity theory; classical spacetime; homogeneity; isotropy; axiomatisation,relativity,[2]
87470,"influence of electrochemical anticorrosion technology on concrete structure and performance. in order to solve the problem of steel bar desensitization in island concrete, which leads to steel bar corrosion and greatly reduces the service life of structures, this work studies the in+uence of ele...",,durability; corrosion; design; bars,[1]
26059,"invariance of plurigenera fails in positive and mixed characteristic. we construct families of elliptic surface pairs with terminal singularities over a dvr of positive or mixed characteristic -> specr, such that p-m > p-m for all sufficiently divisible m > 0. in particular, this shows that ...",,surfaces; varieties,[2]
19163,"in situ generated dbu•hf acts as a fluorinating agent in a hexafluoroisobutylation tandem reaction: an effective route to 5,5,5,5′,5′,5′-hexafluoroleucine. we report the direct incorporation of the hexafluoroisobutyl group on a chiral glycine schiff base complex mediated by 1,8-diazabicyclo[5.4....",fluoroalkylation; fluorination; amino acids; stereoselective synthesis; synthetic methods,"alpha-amino-acids; functionalized 1,1-difluoro-1-alkenes; organolithium reagents; proteolytic stability; asymmetric-synthesis; gem-difluoroalkenes; biological-activity; concise synthesis; chemistry; homologation",[1]
...,...,...,...,...
63543,"boosting bismuth complexation for targeted α-therapy applications with the mesocyclic chelating agent aazta. targeted alpha therapy is a promising tool in the therapy of cancer. the radionuclide bi-213 shows favourable physical properties for this application, but the fast and stable chela...",aazta; bioconjugation; bismuth; octreotate; targeted alpha therapy,neuroendocrine tumors; x-ray; ligands; pet; equilibrium; stability; analogs; ga-68; sc-44; nmr,[1]
61432,"a data-driven obe magnetic interference compensation method. aeromagnetic compensation is a technology used to reduce aircraft magnetic interference, which plays an important role in aeromagnetic surveys. in addition to maneuvering interferences, the onboard electronic interference has been pr...",aeromagnetic survey; obe interference compensation; lstm network,model; networks,[1]
12389,"allosteric activation of 15-lipoxygenase-1 by boswellic acid induces the lipid mediator class switch to promote resolution of inflammation. specialized pro-resolving mediators , primarily produced in innate immune cells, exert crucial bioactions for resolving inflammation. among various lipoxyg...",boswellic acid; inflammation; lipid mediators; lipoxygenases,leukotriene biosynthesis; 5-lipoxygenase-activating protein; in-vitro; disease; calcium; inhibitors; biology; domain; vivo,[1]
10816,"microbiota community structure and interaction networks within dermacentor silvarum, ixodes persulcatus, and haemaphysalis concinna. simple summary ticks pose a threat to humans and animals. however, microbial interactions on ticks are underappreciated. complex microbial interactions shape micro...",tick; microbiota; microbial 

In [ ]:
# - hariç tüm noktalama işaretleri kaldırıldı. örnek olarak push-pull gibi yerlerde - kaldırılınca pushpull olmaması için
for column in ["Abstract", "Author Keywords", "Keywords Plus"]:
  df[column] = df[column].apply(lambda x: ''.join(char for char in x if (char not in string.punctuation) or (char == "-")))

df

,Abstract,Author Keywords,Keywords Plus,label Research Areas
93269,atypical ageing defect in pinot blanc wines a comparison between organic and conventional production management systems background atypical ageing is an aroma defect that occurs in white wines and entails a loss of varietal aromas as well as scents of wet mop shoe polish and dish rag 2-aminoac...,atypical ageing pinot blanc uhplc-hrms aroma fault 2-aminoacetophenone organic wine,biogenic-amines off-flavor grape glutathione tryptophan 2-aminoacetophenone proline stability solutes plants,[1]
57674,investigations of isotropy and homogeneity of spacetime in first-order logic we investigate the logical connection between isotropy homogeneity of space and homogeneity of time within a general axiomatic framework we show that isotropy not only entails homogeneity of space but also in certain ...,first-order logic relativity theory classical spacetime homogeneity isotropy axiomatisation,relativity,[2]
87470,influence of electrochemical anticorrosion technology on concrete structure and performance in order to solve the problem of steel bar desensitization in island concrete which leads to steel bar corrosion and greatly reduces the service life of structures this work studies the inuence of electro...,,durability corrosion design bars,[1]
26059,invariance of plurigenera fails in positive and mixed characteristic we construct families of elliptic surface pairs with terminal singularities over a dvr of positive or mixed characteristic - specr such that p-m p-m for all sufficiently divisible m 0 in particular this shows that invaria...,,surfaces varieties,[2]
19163,in situ generated dbu•hf acts as a fluorinating agent in a hexafluoroisobutylation tandem reaction an effective route to 5555′5′5′-hexafluoroleucine we report the direct incorporation of the hexafluoroisobutyl group on a chiral glycine schiff base complex mediated by 18-diazabicyclo540undec-7-en...,fluoroalkylation fluorination amino acids stereoselective synthesis synthetic methods,alpha-amino-acids functionalized 11-difluoro-1-alkenes organolithium reagents proteolytic stability asymmetric-synthesis gem-difluoroalkenes biological-activity concise synthesis chemistry homologation,[1]
...,...,...,...,...
63543,boosting bismuth complexation for targeted α-therapy applications with the mesocyclic chelating agent aazta targeted alpha therapy is a promising tool in the therapy of cancer the radionuclide bi-213 shows favourable physical properties for this application but the fast and stable chelatio...,aazta bioconjugation bismuth octreotate targeted alpha therapy,neuroendocrine tumors x-ray ligands pet equilibrium stability analogs ga-68 sc-44 nmr,[1]
61432,a data-driven obe magnetic interference compensation method aeromagnetic compensation is a technology used to reduce aircraft magnetic interference which plays an important role in aeromagnetic surveys in addition to maneuvering interferences the onboard electronic interference has been proven...,aeromagnetic survey obe interference compensation lstm network,model networks,[1]
12389,allosteric activation of 15-lipoxygenase-1 by boswellic acid induces the lipid mediator class switch to promote resolution of inflammation specialized pro-resolving mediators primarily produced in innate immune cells exert crucial bioactions for resolving inflammation among various lipoxygenas...,boswellic acid inflammation lipid mediators lipoxygenases,leukotriene biosynthesis 5-lipoxygenase-activating protein in-vitro disease calcium inhibitors biology domain vivo,[1]
10816,microbiota community structure and interaction networks within dermacentor silvarum ixodes persulcatus and haemaphysalis concinna simple summary ticks pose a threat to humans and animals however microbial interactions on ticks are underappreciated complex microbial interactions shape microbial c...,tick microbiota microbial interactions symbiont bacteria 16s rrna gene,bacterial diversity tick infection acari tra

In [ ]:
# - yerine boşluk gelecek
for column in ["Abstract", "Author Keywords", "Keywords Plus"]:
  df[column] = df[column].apply(lambda x: x.replace("-", " "))
df

,Abstract,Author Keywords,Keywords Plus,label Research Areas
93269,atypical ageing defect in pinot blanc wines a comparison between organic and conventional production management systems background atypical ageing is an aroma defect that occurs in white wines and entails a loss of varietal aromas as well as scents of wet mop shoe polish and dish rag 2 aminoac...,atypical ageing pinot blanc uhplc hrms aroma fault 2 aminoacetophenone organic wine,biogenic amines off flavor grape glutathione tryptophan 2 aminoacetophenone proline stability solutes plants,[1]
57674,investigations of isotropy and homogeneity of spacetime in first order logic we investigate the logical connection between isotropy homogeneity of space and homogeneity of time within a general axiomatic framework we show that isotropy not only entails homogeneity of space but also in certain ...,first order logic relativity theory classical spacetime homogeneity isotropy axiomatisation,relativity,[2]
87470,influence of electrochemical anticorrosion technology on concrete structure and performance in order to solve the problem of steel bar desensitization in island concrete which leads to steel bar corrosion and greatly reduces the service life of structures this work studies the inuence of electro...,,durability corrosion design bars,[1]
26059,invariance of plurigenera fails in positive and mixed characteristic we construct families of elliptic surface pairs with terminal singularities over a dvr of positive or mixed characteristic specr such that p m p m for all sufficiently divisible m 0 in particular this shows that invaria...,,surfaces varieties,[2]
19163,in situ generated dbu•hf acts as a fluorinating agent in a hexafluoroisobutylation tandem reaction an effective route to 5555′5′5′ hexafluoroleucine we report the direct incorporation of the hexafluoroisobutyl group on a chiral glycine schiff base complex mediated by 18 diazabicyclo540undec 7 en...,fluoroalkylation fluorination amino acids stereoselective synthesis synthetic methods,alpha amino acids functionalized 11 difluoro 1 alkenes organolithium reagents proteolytic stability asymmetric synthesis gem difluoroalkenes biological activity concise synthesis chemistry homologation,[1]
...,...,...,...,...
63543,boosting bismuth complexation for targeted α therapy applications with the mesocyclic chelating agent aazta targeted alpha therapy is a promising tool in the therapy of cancer the radionuclide bi 213 shows favourable physical properties for this application but the fast and stable chelatio...,aazta bioconjugation bismuth octreotate targeted alpha therapy,neuroendocrine tumors x ray ligands pet equilibrium stability analogs ga 68 sc 44 nmr,[1]
61432,a data driven obe magnetic interference compensation method aeromagnetic compensation is a technology used to reduce aircraft magnetic interference which plays an important role in aeromagnetic surveys in addition to maneuvering interferences the onboard electronic interference has been proven...,aeromagnetic survey obe interference compensation lstm network,model networks,[1]
12389,allosteric activation of 15 lipoxygenase 1 by boswellic acid induces the lipid mediator class switch to promote resolution of inflammation specialized pro resolving mediators primarily produced in innate immune cells exert crucial bioactions for resolving inflammation among various lipoxygenas...,boswellic acid inflammation lipid mediators lipoxygenases,leukotriene biosynthesis 5 lipoxygenase activating protein in vitro disease calcium inhibitors biology domain vivo,[1]
10816,microbiota community structure and interaction networks within dermacentor silvarum ixodes persulcatus and haemaphysalis concinna simple summary ticks pose a threat to humans and animals however microbial interactions on ticks are underappreciated complex microbial interactions shape microbial c...,tick microbiota microbial interactions symbiont bacteria 16s rrna gene,bacterial diversity tick infection acari trans

In [ ]:
# roma rakamlarını kaldır
for column in ["Abstract", "Author Keywords", "Keywords Plus"]:
  df[column] = df[column].apply(lambda x: re.sub(r'\b[IVXLCDM]+\b', '', x))
df

,Abstract,Author Keywords,Keywords Plus,label Research Areas
93269,atypical ageing defect in pinot blanc wines a comparison between organic and conventional production management systems background atypical ageing is an aroma defect that occurs in white wines and entails a loss of varietal aromas as well as scents of wet mop shoe polish and dish rag 2 aminoac...,atypical ageing pinot blanc uhplc hrms aroma fault 2 aminoacetophenone organic wine,biogenic amines off flavor grape glutathione tryptophan 2 aminoacetophenone proline stability solutes plants,[1]
57674,investigations of isotropy and homogeneity of spacetime in first order logic we investigate the logical connection between isotropy homogeneity of space and homogeneity of time within a general axiomatic framework we show that isotropy not only entails homogeneity of space but also in certain ...,first order logic relativity theory classical spacetime homogeneity isotropy axiomatisation,relativity,[2]
87470,influence of electrochemical anticorrosion technology on concrete structure and performance in order to solve the problem of steel bar desensitization in island concrete which leads to steel bar corrosion and greatly reduces the service life of structures this work studies the inuence of electro...,,durability corrosion design bars,[1]
26059,invariance of plurigenera fails in positive and mixed characteristic we construct families of elliptic surface pairs with terminal singularities over a dvr of positive or mixed characteristic specr such that p m p m for all sufficiently divisible m 0 in particular this shows that invaria...,,surfaces varieties,[2]
19163,in situ generated dbu•hf acts as a fluorinating agent in a hexafluoroisobutylation tandem reaction an effective route to 5555′5′5′ hexafluoroleucine we report the direct incorporation of the hexafluoroisobutyl group on a chiral glycine schiff base complex mediated by 18 diazabicyclo540undec 7 en...,fluoroalkylation fluorination amino acids stereoselective synthesis synthetic methods,alpha amino acids functionalized 11 difluoro 1 alkenes organolithium reagents proteolytic stability asymmetric synthesis gem difluoroalkenes biological activity concise synthesis chemistry homologation,[1]
...,...,...,...,...
63543,boosting bismuth complexation for targeted α therapy applications with the mesocyclic chelating agent aazta targeted alpha therapy is a promising tool in the therapy of cancer the radionuclide bi 213 shows favourable physical properties for this application but the fast and stable chelatio...,aazta bioconjugation bismuth octreotate targeted alpha therapy,neuroendocrine tumors x ray ligands pet equilibrium stability analogs ga 68 sc 44 nmr,[1]
61432,a data driven obe magnetic interference compensation method aeromagnetic compensation is a technology used to reduce aircraft magnetic interference which plays an important role in aeromagnetic surveys in addition to maneuvering interferences the onboard electronic interference has been proven...,aeromagnetic survey obe interference compensation lstm network,model networks,[1]
12389,allosteric activation of 15 lipoxygenase 1 by boswellic acid induces the lipid mediator class switch to promote resolution of inflammation specialized pro resolving mediators primarily produced in innate immune cells exert crucial bioactions for resolving inflammation among various lipoxygenas...,boswellic acid inflammation lipid mediators lipoxygenases,leukotriene biosynthesis 5 lipoxygenase activating protein in vitro disease calcium inhibitors biology domain vivo,[1]
10816,microbiota community structure and interaction networks within dermacentor silvarum ixodes persulcatus and haemaphysalis concinna simple summary ticks pose a threat to humans and animals however microbial interactions on ticks are underappreciated complex microbial interactions shape microbial c...,tick microbiota microbial interactions symbiont bacteria 16s rrna gene,bacterial diversity tick infection acari trans

In [ ]:
# yazıyla yazılmış sayıları kaldır. örneğin one, two gibi
remove_text_numbers = lambda text: re.sub(r'\b(?:zero|one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion)\b', '', text, flags=re.IGNORECASE)
for column in ["Abstract", "Author Keywords", "Keywords Plus"]:
  df[column] = df[column].apply(remove_text_numbers)
df

,Abstract,Author Keywords,Keywords Plus,label Research Areas
93269,atypical ageing defect in pinot blanc wines a comparison between organic and conventional production management systems background atypical ageing is an aroma defect that occurs in white wines and entails a loss of varietal aromas as well as scents of wet mop shoe polish and dish rag 2 aminoac...,atypical ageing pinot blanc uhplc hrms aroma fault 2 aminoacetophenone organic wine,biogenic amines off flavor grape glutathione tryptophan 2 aminoacetophenone proline stability solutes plants,[1]
57674,investigations of isotropy and homogeneity of spacetime in first order logic we investigate the logical connection between isotropy homogeneity of space and homogeneity of time within a general axiomatic framework we show that isotropy not only entails homogeneity of space but also in certain ...,first order logic relativity theory classical spacetime homogeneity isotropy axiomatisation,relativity,[2]
87470,influence of electrochemical anticorrosion technology on concrete structure and performance in order to solve the problem of steel bar desensitization in island concrete which leads to steel bar corrosion and greatly reduces the service life of structures this work studies the inuence of electro...,,durability corrosion design bars,[1]
26059,invariance of plurigenera fails in positive and mixed characteristic we construct families of elliptic surface pairs with terminal singularities over a dvr of positive or mixed characteristic specr such that p m p m for all sufficiently divisible m 0 in particular this shows that invaria...,,surfaces varieties,[2]
19163,in situ generated dbu•hf acts as a fluorinating agent in a hexafluoroisobutylation tandem reaction an effective route to 5555′5′5′ hexafluoroleucine we report the direct incorporation of the hexafluoroisobutyl group on a chiral glycine schiff base complex mediated by 18 diazabicyclo540undec 7 en...,fluoroalkylation fluorination amino acids stereoselective synthesis synthetic methods,alpha amino acids functionalized 11 difluoro 1 alkenes organolithium reagents proteolytic stability asymmetric synthesis gem difluoroalkenes biological activity concise synthesis chemistry homologation,[1]
...,...,...,...,...
63543,boosting bismuth complexation for targeted α therapy applications with the mesocyclic chelating agent aazta targeted alpha therapy is a promising tool in the therapy of cancer the radionuclide bi 213 shows favourable physical properties for this application but the fast and stable chelatio...,aazta bioconjugation bismuth octreotate targeted alpha therapy,neuroendocrine tumors x ray ligands pet equilibrium stability analogs ga 68 sc 44 nmr,[1]
61432,a data driven obe magnetic interference compensation method aeromagnetic compensation is a technology used to reduce aircraft magnetic interference which plays an important role in aeromagnetic surveys in addition to maneuvering interferences the onboard electronic interference has been proven...,aeromagnetic survey obe interference compensation lstm network,model networks,[1]
12389,allosteric activation of 15 lipoxygenase 1 by boswellic acid induces the lipid mediator class switch to promote resolution of inflammation specialized pro resolving mediators primarily produced in innate immune cells exert crucial bioactions for resolving inflammation among various lipoxygenas...,boswellic acid inflammation lipid mediators lipoxygenases,leukotriene biosynthesis 5 lipoxygenase activating protein in vitro disease calcium inhibitors biology domain vivo,[1]
10816,microbiota community structure and interaction networks within dermacentor silvarum ixodes persulcatus and haemaphysalis concinna simple summary ticks pose a threat to humans and animals however microbial interactions on ticks are underappreciated complex microbial interactions shape microbial c...,tick microbiota microbial interactions symbiont bacteria 16s rrna gene,bacterial diversity tick infection acari trans

In [ ]:
def remove_length_units(text):
    length_units_pattern = re.compile(r'\b(?:cm|m|m2|m3|cm2|km2|km|em|pm|zm|nm|mm|dam|μm|dm|pm|hm|ft2|gm|kg|g|mg|m²|ft²)\b', re.IGNORECASE)

    text_without_length_units = re.sub(length_units_pattern, '', text)

    return text_without_length_units

for column in ["Abstract", "Author Keywords", "Keywords Plus"]:
  df[column] = df[column].apply(remove_length_units)
df

,Abstract,Author Keywords,Keywords Plus,label Research Areas
93269,atypical ageing defect in pinot blanc wines a comparison between organic and conventional production management systems background atypical ageing is an aroma defect that occurs in white wines and entails a loss of varietal aromas as well as scents of wet mop shoe polish and dish rag 2 aminoac...,atypical ageing pinot blanc uhplc hrms aroma fault 2 aminoacetophenone organic wine,biogenic amines off flavor grape glutathione tryptophan 2 aminoacetophenone proline stability solutes plants,[1]
57674,investigations of isotropy and homogeneity of spacetime in first order logic we investigate the logical connection between isotropy homogeneity of space and homogeneity of time within a general axiomatic framework we show that isotropy not only entails homogeneity of space but also in certain ...,first order logic relativity theory classical spacetime homogeneity isotropy axiomatisation,relativity,[2]
87470,influence of electrochemical anticorrosion technology on concrete structure and performance in order to solve the problem of steel bar desensitization in island concrete which leads to steel bar corrosion and greatly reduces the service life of structures this work studies the inuence of electro...,,durability corrosion design bars,[1]
26059,invariance of plurigenera fails in positive and mixed characteristic we construct families of elliptic surface pairs with terminal singularities over a dvr of positive or mixed characteristic specr such that p p for all sufficiently divisible 0 in particular this shows that invariance...,,surfaces varieties,[2]
19163,in situ generated dbu•hf acts as a fluorinating agent in a hexafluoroisobutylation tandem reaction an effective route to 5555′5′5′ hexafluoroleucine we report the direct incorporation of the hexafluoroisobutyl group on a chiral glycine schiff base complex mediated by 18 diazabicyclo540undec 7 en...,fluoroalkylation fluorination amino acids stereoselective synthesis synthetic methods,alpha amino acids functionalized 11 difluoro 1 alkenes organolithium reagents proteolytic stability asymmetric synthesis gem difluoroalkenes biological activity concise synthesis chemistry homologation,[1]
...,...,...,...,...
63543,boosting bismuth complexation for targeted α therapy applications with the mesocyclic chelating agent aazta targeted alpha therapy is a promising tool in the therapy of cancer the radionuclide bi 213 shows favourable physical properties for this application but the fast and stable chelatio...,aazta bioconjugation bismuth octreotate targeted alpha therapy,neuroendocrine tumors x ray ligands pet equilibrium stability analogs ga 68 sc 44 nmr,[1]
61432,a data driven obe magnetic interference compensation method aeromagnetic compensation is a technology used to reduce aircraft magnetic interference which plays an important role in aeromagnetic surveys in addition to maneuvering interferences the onboard electronic interference has been proven...,aeromagnetic survey obe interference compensation lstm network,model networks,[1]
12389,allosteric activation of 15 lipoxygenase 1 by boswellic acid induces the lipid mediator class switch to promote resolution of inflammation specialized pro resolving mediators primarily produced in innate immune cells exert crucial bioactions for resolving inflammation among various lipoxygenas...,boswellic acid inflammation lipid mediators lipoxygenases,leukotriene biosynthesis 5 lipoxygenase activating protein in vitro disease calcium inhibitors biology domain vivo,[1]
10816,microbiota community structure and interaction networks within dermacentor silvarum ixodes persulcatus and haemaphysalis concinna simple summary ticks pose a threat to humans and animals however microbial interactions on ticks are underappreciated complex microbial interactions shape microbial c...,tick microbiota microbial interactions symbiont bacteria 16s rrna gene,bacterial diversity tick infection acari transmis

In [ ]:
# sayıları tekrar kaldır
for column in ["Abstract", "Author Keywords", "Keywords Plus"]:
  df[column] = df[column].apply(lambda x: ''.join(char for char in x if (char not in string.punctuation)))
df

In [ ]:
# sayı içeren stringlerdeki sayıları sil. co2 yerine co
def sayilari_sil(metin):
    temizlenmis_metin = re.sub(r'\d', '', metin)
    return temizlenmis_metin
for column in ["Abstract", "Author Keywords", "Keywords Plus"]:
  df[column] = df[column].apply(sayilari_sil)

df

In [ ]:
# fazla boşluk kaldırma (baş/son boşlukları ve ortalardaki boşlukları teke indirme)
for column in ["Abstract", "Author Keywords", "Keywords Plus"]:
  df[column] = df[column].apply(lambda x: x.strip())
  df[column] = df[column].apply(lambda x: re.sub(' +', ' ', x))
df

## 4.Remove StopWords (NLTK)

In [ ]:
for column in ["Abstract", "Author Keywords", "Keywords Plus"]:
  df[column] = df[column].apply(lambda text: ' '.join([word for word in text.split() if word not in stopwords.words("english")]))
df

,Abstract,Author Keywords,Keywords Plus,label Research Areas
93269,atypical ageing defect pinot blanc wines comparison organic conventional production management systems background atypical ageing aroma defect occurs white wines entails loss varietal aromas well scents wet mop shoe polish dish rag aminoacetophenone degradation product indole acetic acid describ...,atypical ageing pinot blanc uhplc hrms aroma fault aminoacetophenone organic wine,biogenic amines flavor grape glutathione tryptophan aminoacetophenone proline stability solutes plants,[1]
57674,investigations isotropy homogeneity spacetime first order logic investigate logical connection isotropy homogeneity space homogeneity time within general axiomatic framework show isotropy entails homogeneity space also certain cases homogeneity time turn homogeneity time implies homogeneity spac...,first order logic relativity theory classical spacetime homogeneity isotropy axiomatisation,relativity,[2]
87470,influence electrochemical anticorrosion technology concrete structure performance order solve problem steel bar desensitization island concrete leads steel bar corrosion greatly reduces service life structures work studies inuence electrochemical corrosion prevention technology structure perform...,,durability corrosion design bars,[1]
26059,invariance plurigenera fails positive mixed characteristic construct families elliptic surface pairs terminal singularities dvr positive mixed characteristic specr p p sufficiently divisible particular shows invariance sufficiently divisible plurigenera follow mmp abundance conjectures,,surfaces varieties,[2]
19163,situ generated dbu•hf acts fluorinating agent hexafluoroisobutylation tandem reaction effective route ′′′ hexafluoroleucine report direct incorporation hexafluoroisobutyl group chiral glycine schiff base complex mediated diazabicycloundec ene fluoroalkylation involves hexafluoropropane reagent g...,fluoroalkylation fluorination amino acids stereoselective synthesis synthetic methods,alpha amino acids functionalized difluoro alkenes organolithium reagents proteolytic stability asymmetric synthesis gem difluoroalkenes biological activity concise synthesis chemistry homologation,[1]
...,...,...,...,...
63543,boosting bismuth complexation targeted α therapy applications mesocyclic chelating agent aazta targeted alpha therapy promising tool therapy cancer radionuclide bi shows favourable physical properties application fast stable chelation metal ion remains challenging herein demonstrate mesocyclic c...,aazta bioconjugation bismuth octreotate targeted alpha therapy,neuroendocrine tumors x ray ligands pet equilibrium stability analogs ga sc nmr,[1]
61432,data driven obe magnetic interference compensation method aeromagnetic compensation technology used reduce aircraft magnetic interference plays important role aeromagnetic surveys addition maneuvering interferences onboard electronic interference proven significant part aircraft interference mus...,aeromagnetic survey obe interference compensation lstm network,model networks,[1]
12389,allosteric activation lipoxygenase boswellic acid induces lipid mediator class switch promote resolution inflammation specialized pro resolving mediators primarily produced innate immune cells exert crucial bioactions resolving inflammation among various lipoxygenases lox key spm biosynthesis ce...,boswellic acid inflammation lipid mediators lipoxygenases,leukotriene biosynthesis lipoxygenase activating protein vitro disease calcium inhibitors biology domain vivo,[1]
10816,microbiota community structure interaction networks within dermacentor silvarum ixodes persulcatus haemaphysalis concinna simple summary ticks pose threat humans animals however microbial interactions ticks underappreciated complex microbial interactions shape microbial communities limited infor...,tick microbiota microbial interactions symbiont bacteria rrna gene,bacterial diversity tick infection acari transmission wolbachia e

In [ ]:
# bir özette en fazla kaç kelime var
def counter(text):
    tokens = []
    for token in text.split():
        tokens.append(token)
    return len(tokens)

df["token_sayilari"] = df["Abstract"].apply(counter)
df["token_sayilari"].describe()

count    81527.000000
mean       131.732702
std         48.610343
min          7.000000
25%        101.000000
50%        130.000000
75%        160.000000
max        921.000000
Name: token_sayilari, dtype: float64

In [ ]:
# özet uzunluğu aykırı değer olan yani fazla olanları almıyoruz
df = df[(df["token_sayilari"] <= 400) and (df["token_sayilari"] >= 400)]